In [1]:
#=======================================================================
# 예제 : sLLM을 이용한 chat봇 예제 
#
#=======================================================================
import torch
import transformers
from peft import PeftModel
import time

from myutils import GPU_info, seed_everything
device = GPU_info()
#seed 설정
SEED = 111
seed_everything(SEED)

#---------------------------------------------------------------------
# param 설정
lora_weights:str = '../data11/model/LLM/beomi/KoAlpaca-Polyglot-5.8B-LoRA-qa-2/'
model_path:str ='../data11/model/LLM/beomi/KoAlpaca-Polyglot-5.8B/'

uselora_weight = True # Lora 사용하는 경우 True
load_8bit = True
#---------------------------------------------------------------------

start_time = time.time()

# tokenizer 로딩
tokenizer = transformers.AutoTokenizer.from_pretrained( model_path)

# 원본 모델 로딩
model = transformers.AutoModelForCausalLM.from_pretrained(model_path, load_in_8bit=load_8bit, torch_dtype=torch.float16, device_map="auto")

if uselora_weight:
    model = PeftModel.from_pretrained(model, lora_weights, torch_dtype=torch.float16) # loRA 모델 로딩

if not load_8bit:
    model.half()
    
print(model.eval())

end_time = time.time() - start_time
print("time: {:.2f} ms\n".format(end_time * 1000)) 



===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
bin /MOCOMSYS/anaconda3/envs/bong/lib/python3.9/site-packages/bitsandbytes/libbitsandbytes_cuda113.so
CUDA SETUP: CUDA runtime path found: /MOCOMSYS/anaconda3/envs/bong/lib/libcudart.so.11.0
CUDA SETUP: Highest compute capability among GPUs detected: 8.0
CUDA SETUP: Detected CUDA version 113
CUDA SETUP: Loading binary /MOCOMSYS/anaconda3/envs/bong/lib/python3.9/site-packages/bitsandbytes/libbitsandbytes_cuda113.so...


/MOCOMSYS/anaconda3/envs/bong/lib/python3.9/site-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: Found duplicate ['libcudart.so', 'libcudart.so.11.0', 'libcudart.so.12.0'] files: {PosixPath('/MOCOMSYS/anaconda3/envs/bong/lib/libcudart.so.11.0'), PosixPath('/MOCOMSYS/anaconda3/envs/bong/lib/libcudart.so')}.. We'll flip a coin and try one of these, in order to fail forward.
Either way, this might cause trouble in the future:
If you get `CUDA error: invalid device function` errors, the above might be the cause and the solution is to make sure only one ['libcudart.so', 'libcudart.so.11.0', 'libcudart.so.12.0'] in the paths that we search based on your env.
  warn(msg)


True
device: cuda:0
cuda index: 0
gpu 개수: 1
graphic name: NVIDIA A30


The model weights are not tied. Please use the `tie_weights` method before using the `infer_auto_device` function.


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): GPTNeoXForCausalLM(
      (gpt_neox): GPTNeoXModel(
        (embed_in): Embedding(30080, 4096)
        (layers): ModuleList(
          (0-27): 28 x GPTNeoXLayer(
            (input_layernorm): LayerNorm((4096,), eps=1e-05, elementwise_affine=True)
            (post_attention_layernorm): LayerNorm((4096,), eps=1e-05, elementwise_affine=True)
            (attention): GPTNeoXAttention(
              (rotary_emb): RotaryEmbedding()
              (query_key_value): Linear8bitLt(
                in_features=4096, out_features=12288, bias=True
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.05, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=4096, out_features=8, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=8, out_features=12288, bias=False)
              

In [4]:
import torch
import time
from transformers import GenerationConfig

eos_str = tokenizer.decode(tokenizer.eos_token_id)

def run_query_loop():
    while True:
        try:
            handle_query()
        except KeyboardInterrupt:
            return
        
def handle_query(max_new_tokens:int=256):
    
    query = input("질문:")
    
    start_time = time.time()
    #prompt = query
    #prompt = f"<mocoquery>{query}</mocoquery>\n\n<answer>"
    #prompt = f"### 질문: {input_text}\n\n### 맥락: {context}\n\n### 답변:" if context else f"### 질문: {input_text}\n\n### 답변:"
    #prompt = f"### 질문 : 간략히 답변해줘.{query}\r\n###답변:"
    prompt = f"### 질문: {query}\n\n### 답변:"
    
    #print(prompt)
    
    # config 설정
    generation_config = GenerationConfig(
            temperature=0.5,
            top_p=0.75,
            top_k=40,
            num_beams=1,
            bos_token_id=tokenizer.bos_token_id,  # 시작토큰 
            eos_token_id=tokenizer.eos_token_id,  # end 토큰
            pad_token_id=tokenizer.pad_token_id   # padding 토큰
    )
    
    # 프롬프트 tokenizer 
    inputs = tokenizer(prompt, return_tensors="pt").to(device)
    input_ids = inputs["input_ids"]
    #print(input_ids)

    # Without streaming
    # generate 처리
    with torch.no_grad():
        generation_output = model.generate(
            input_ids=input_ids,
            generation_config=generation_config,
            return_dict_in_generate=True,
            output_scores=False,
            max_new_tokens=max_new_tokens,
        )

    # 출력
    s = generation_output.sequences[0]
    output = tokenizer.decode(s)

    end_time = time.time() - start_time
    print("time: {:.2f} ms\n".format(end_time * 1000)) 
    print(output.replace(eos_str, ''))
    print()


In [ ]:
run_query_loop()

질문: 모코엠시스란


time: 11229.52 ms

### 질문: 모코엠시스란

### 답변:모코엠시스는'mocha'라는 단어의 일본식 발음으로, '모카'라는 커피 원두에 '바나나'라는 과일을 섞어 만든 디저트입니다. 이는 60년대에 일본에서 유행하던 커피 + 바나나 조합으로 탄생했습니다. 이후, 전 세계적으로 유행하게 된 디저트이며, 특히 미국과 이탈리아에서 유명합니다. 모코엠시스는 초콜릿과 생크림을 위에 얹어 만든 달콤하고 부드러운 디저트입니다. 



질문: 모코엠시스는 어떤 회사인가요?


time: 2254.89 ms

### 질문: 모코엠시스는 어떤 회사인가요?

### 답변:모코엠시스는 휴대용 무선 충전기를 개발하는 회사입니다.



질문: 모코엠시스 업체는 어떤 일을 하나요?


time: 12356.82 ms

### 질문: 모코엠시스 업체는 어떤 일을 하나요?

### 답변:모코엠시스는 휴대용 유전자 분석 서비스를 제공하는 국내 유일의 유전자 분석 기업입니다. 모코엠시스는 유전자 분석을 통해 질병, 건강, 성격 등 다양한 분야에서 활용될 수 있는 기술을 개발하고 있습니다. 모코엠시스는 휴대용 유전자 분석 시스템을 구축하여, 이를 통해 다양한 질병, 건강, 성격 등을 분석하여 서비스를 제공하고 있습니다. 최근에는 코로나19 진단 키트에도 모코엠시스 기술이 활용되어 효과적인 결과를 내고 있습니다. 

